# Elevrad, toporad and stoporad

Ensure that the conversion of the 3 functions to Python match with the IPW versions

In [ ]:
import os
import glob
import netCDF4 as nc
import pandas as pd
import numpy as np
import subprocess as sp
from spatialnc import ipw
from spatialnc.topo import get_topo_stats
from topocalc.shade import shade
import matplotlib.pyplot as plt
import seaborn as sns

from smrf.data import loadTopo
from smrf.envphys.sunang import sunang
from smrf.envphys.albedo import albedo
from smrf.envphys.solar import toporad, irradiance

sns.set_context('poster')

In [ ]:
# Topo, create an IPW image from topo, sky view factor and terrain config factor

# Load the netcdf dem and get all the geo coords for IPW
topo_path = '../tests/Lakes/topo/topo.nc'

topo_config = {
    'filename': topo_path,
    'northern_hemisphere': True,
    'gradient_method': 'gradient_d8',
    'sky_view_factor_angles': 72
}
topo = loadTopo.Topo(topo_config, tempDir='.')
dem = topo.dem

ts = get_topo_stats(topo_path)

csys = 'UTM'
nbits = 16

# Create the IPW image for dem
ipw_image = ipw.IPW()
ipw_image.new_band(dem)
ipw_image.add_geo_hdr(
    coordinates=[ts['u'], ts['v']],
    d=[ts['dv'], ts['du']],
    units=ts['units'],
    csys=csys
)
ipw_image.write('lakes_dem.ipw', nbits=nbits)


# Create the IPW image for sky view factor
ipw_image = ipw.IPW()
ipw_image.new_band(topo.sky_view_factor)
ipw_image.add_geo_hdr(
    coordinates=[ts['u'], ts['v']],
    d=[ts['dv'], ts['du']],
    units=ts['units'],
    csys=csys
)
ipw_image.write('svf.ipw', nbits=nbits)

# Create the IPW image for terrain configuration factor
ipw_image = ipw.IPW()
ipw_image.new_band(topo.terrain_config_factor)
ipw_image.add_geo_hdr(
    coordinates=[ts['u'], ts['v']],
    d=[ts['dv'], ts['du']],
    units=ts['units'],
    csys=csys
)
ipw_image.write('tcf.ipw', nbits=nbits)


In [ ]:
# Parameters for elevrad, toporad and stoporad

date_time = pd.to_datetime('2/15/1990 20:30')
date_time = date_time.tz_localize('UTC')

tau_elevation = 100
tau = 0.2
omega = 0.85
scattering_factor = 0.3
surface_albedo = 0.5
solar_irradiance = irradiance.direct_solar_irradiance(
    date_time, w=[0.28, 2.8])

cosz, azimuth, rad_vec = sunang(
    date_time,
    topo.basin_lat,
    topo.basin_long)

illum_ang = shade(
    topo.sin_slope,
    topo.aspect,
    azimuth,
    cosz)

alb = albedo(10 * np.ones_like(dem), illum_ang, 100, 700)

# Create the IPW image for illumination angle
ipw_image = ipw.IPW()
ipw_image.new_band(illum_ang)
ipw_image.add_geo_hdr(
    coordinates=[ts['u'], ts['v']],
    d=[ts['dv'], ts['du']],
    units=ts['units'],
    csys=csys
)

ipw_image.write('illum_angle.ipw', nbits=nbits)

# Create the IPW image for albedo
ipw_image = ipw.IPW()
ipw_image.new_band(alb[0])
ipw_image.add_geo_hdr(
    coordinates=[ts['u'], ts['v']],
    d=[ts['dv'], ts['du']],
    units=ts['units'],
    csys=csys
)

ipw_image.write('albedo.ipw', nbits=nbits)

In [ ]:
# Elevrad
# The only difference is bit resolution noise

# Call IPW elevrad
cmd = f"elevrad -z {tau_elevation} -t {tau} -w {omega} -g {scattering_factor} -r {surface_albedo} -s {solar_irradiance} -u {cosz} -n 16 lakes_dem.ipw > elevrad.ipw"

print(cmd)
visp = sp.Popen(cmd, shell=True)
visp.wait()

elevrad_ipw = ipw.IPW('elevrad.ipw')

# Python elevrad
elevrad_py = toporad.Elevrad(
    dem,
    solar_irradiance,
    cosz)

# Plot
fig, ax = plt.subplots(2, 3, figsize = (30, 20))
fig.subplots_adjust(left=0.02, bottom=0.06, right=0.95, top=0.94, wspace=0.05)
fig.suptitle('ELEVRAD')

# IPW originals
im0 = ax[0, 0].imshow(elevrad_ipw.bands[0].data)
ax[0, 0].set_title('IPW beam')
fig.colorbar(im0, ax=ax[0, 0])

im0 = ax[1, 0].imshow(elevrad_ipw.bands[1].data)
ax[1, 0].set_title('IPW diffuse')
fig.colorbar(im0, ax=ax[1, 0])

# Python elevrad
im0 = ax[0, 1].imshow(elevrad_py.beam)
ax[0, 1].set_title('Python beam')
fig.colorbar(im0, ax=ax[0, 1])

im0 = ax[1, 1].imshow(elevrad_py.diffuse)
ax[1, 1].set_title('Python diffuse')
fig.colorbar(im0, ax=ax[1, 1])

# difference
d = elevrad_ipw.bands[0].data - elevrad_py.beam
var_lim = max([abs(d.min()), abs(d.max())])
var_lim = 1e-4
im2 = ax[0, 2].imshow(d, cmap='RdBu', vmin=-var_lim, vmax=var_lim)
ax[0, 2].set_title('Beam difference, ipw - python')
fig.colorbar(im2, ax=ax[0, 2])

d = elevrad_ipw.bands[1].data - elevrad_py.diffuse
var_lim = max([abs(d.min()), abs(d.max())])
var_lim = 1e-4
im2 = ax[1, 2].imshow(d, cmap='RdBu', vmin=-var_lim, vmax=var_lim)
ax[1, 2].set_title('Diffuse difference, ipw - python')
fig.colorbar(im2, ax=ax[1, 2])

# remove the x and y ticks
for a in ax.flatten():
    a.set_xticks([])
    a.set_yticks([])

In [ ]:
# toporad
# The only difference is bit resolution noise

# IPW toporad
# toporad requires a sun header that has the cosz and azimuth. Since this example hasn't done that, toporad sets cosz to 0.5

cmd = f"mux elevrad.ipw illum_angle.ipw svf.ipw tcf.ipw albedo.ipw | toporad -r {surface_albedo} -d > toporad.ipw"

print(cmd)
visp = sp.Popen(cmd, shell=True)
visp.wait()

toporad_ipw = ipw.IPW('toporad.ipw')

# Python toporad
erad = toporad.Elevrad(dem, solar_irradiance, cosz)

trad_beam, trad_diffuse = toporad.toporad(
    erad.beam,
    erad.diffuse,
    illum_ang,
    topo.sky_view_factor,
    topo.terrain_config_factor,
    0.5,
    surface_albedo=0.5)

# Plot
fig, ax = plt.subplots(2, 3, figsize = (30, 20))
fig.subplots_adjust(left=0.02, bottom=0.06, right=0.95, top=0.94, wspace=0.05)
fig.suptitle('TOPORAD')

# IPW originals
im0 = ax[0, 0].imshow(toporad_ipw.bands[0].data)
ax[0, 0].set_title('IPW beam')
fig.colorbar(im0, ax=ax[0, 0])

im0 = ax[1, 0].imshow(toporad_ipw.bands[1].data)
ax[1, 0].set_title('IPW diffuse')
fig.colorbar(im0, ax=ax[1, 0])

# Python elevrad
im0 = ax[0, 1].imshow(trad_beam)
ax[0, 1].set_title('Python beam')
fig.colorbar(im0, ax=ax[0, 1])

im0 = ax[1, 1].imshow(trad_diffuse)
ax[1, 1].set_title('Python diffuse')
fig.colorbar(im0, ax=ax[1, 1])

# difference
d = toporad_ipw.bands[0].data - trad_beam
var_lim = max([abs(d.min()), abs(d.max())])
var_lim = 1e-4
im2 = ax[0, 2].imshow(d, cmap='RdBu', vmin=-var_lim, vmax=var_lim)
ax[0, 2].set_title('Beam difference, ipw - python')
fig.colorbar(im2, ax=ax[0, 2])

d = toporad_ipw.bands[1].data - trad_diffuse
var_lim = max([abs(d.min()), abs(d.max())])
var_lim = 1e-4
im2 = ax[1, 2].imshow(d, cmap='RdBu', vmin=-var_lim, vmax=var_lim)
ax[1, 2].set_title('Diffuse difference, ipw - python')
fig.colorbar(im2, ax=ax[1, 2])

# remove the x and y ticks
for a in ax.flatten():
    a.set_xticks([])
    a.set_yticks([])

In [ ]:
# clean up IPW images
for fileName in glob.glob("*.ipw"):
  os.remove(fileName)